In [ ]:
%load_ext load_style
%load_style talk.css

# Pandas

In [ ]:
import addutils.toc ; addutils.toc.js(ipy_notebook=True)

In [ ]:
from IPython.display import Image, HTML


**pandas** is a Python package providing fast, flexible, and expressive data structures designed to work with *relational* or *labeled* data. It is a fundamental high-level building block for doing practical, real world data analysis in Python. 

pandas is well suited for:

- Tabular data with heterogeneously-typed columns, as in an SQL table or Excel spreadsheet
- Ordered and unordered (not necessarily fixed-frequency) time series data.
- Arbitrary matrix data (homogeneously typed or heterogeneous) with row and column labels
- Any other form of observational / statistical data sets. The data actually need not be labeled at all to be placed into a pandas data structure


Key features:  
    
- Easy handling of **missing data**
- **Size mutability**: columns can be inserted and deleted from DataFrame and higher dimensional objects
- Automatic and explicit **data alignment**: objects can be explicitly aligned to a set of labels, or the data can be aligned automatically
- Powerful, flexible **group by functionality** to perform split-apply-combine operations on data sets
- Intelligent label-based **slicing, fancy indexing, and subsetting** of large data sets
- Intuitive **merging and joining** data sets
- Flexible **reshaping and pivoting** of data sets
- **Hierarchical labeling** of axes
- Robust **IO tools** for loading data from flat files, Excel files, databases, and HDF5
- **Time series functionality**: date range generation and frequency conversion, moving window statistics, moving window linear regressions, date shifting and lagging, etc.

In [ ]:
HTML('<iframe src=http://pandas.pydata.org/index.html width=900 height=350></iframe>')

The conventional way to import pandas is: 

```python
import pandas as pd
```

In [ ]:
import pandas as pd

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

some options setting 

In [ ]:
pd.set_option("display.width", 80)
# toggle the line below that if one doesnt want DataFrames displayed as HTML tables
#pd.set_option("notebook_repr_html", False) 
pd.set_option("notebook_repr_html", True) 

Pandas's data structures and functionalities will be familiar to R users, there's a section on Pandas's website where 
Wes McKinney gives some translation of common idioms / operations between R and Pandas 

In [ ]:
HTML('<iframe src=http://pandas.pydata.org/pandas-docs/stable/comparison_with_r.html#compare-with-r width=900 height=350></iframe>')

## Pandas data structures

### Series


A **Series** is a single vector of data values (think a NumPy array with shape N or (N,1)) with an **index** that labels each element in the vector.

#### Series constructions

In [ ]:
a = pd.Series(np.random.normal(0,1,(10,)))

In [ ]:
a

In [ ]:
a.index

In [ ]:
a.values

#### You can define your own **index**

In [ ]:
a = pd.Series(np.random.normal(0,1,(10,)), index=np.arange(1,11))

In [ ]:
a

In [ ]:
a = pd.Series(np.random.normal(0,1,5), \
              index=['a','b','c','d','e'], name='my series')

In [ ]:
a

Pandas objects expose some powerful, **high level plotting functions** (built on top of Matplotlib)

In [ ]:
plot = a.plot(kind='bar', rot=0, color='r', title=a.name)

In [ ]:
f, ax = plt.subplots()
bars = ax.bar(np.arange(len(a)), a.values, color='w', edgecolor='k', align='center', hatch='///')
ax.set_xticks(np.arange(len(a)))
ax.set_xlim(-0.5, len(a)-0.5)
ax.set_xticklabels(a.index, fontsize=16)
ax.set_title(a.name, fontsize=16)

#### Series indexing

Selecting from a Series is easy, using the corresponding index key (like a dict)

In [ ]:
a

In [ ]:
a['c']

slices are permitted 

In [ ]:
a['a':'c'] ### Note the difference with standard Python / Numpy positional, integer indexing

In [ ]:
a['c':]

deleting an element 

In [ ]:
a.drop('d')

Adding an element is (to my knowledge) not straightforward

In [ ]:
a = a.drop('d')

In [ ]:
a

In [ ]:
a.append(pd.Series({'d':5}))

Mathematical operations involving two series will perform operations by *aligning indices*.

1. The union of all indices is created
2. The mathematical operation is performed on matching indices. 

Indices that do not match are given the value NaN (not a number), and values are computed for all unique pairs of repeated indices.

In [ ]:
s1 = pd.Series(np.arange(1.0,4.0),index=['a','b','c'])
s2 = pd.Series(np.arange(1.0,4.0),index=['b','c','d'])

In [ ]:
s3 = s1 + s2

In [ ]:
s3

NaNs are ignored in all operations 

In [ ]:
s3.mean()

You can drop them from the Series

In [ ]:
s4 = s3.dropna()

In [ ]:
s4

Or use the `fillna` method to replace them by a value

In [ ]:
s3.fillna(-999)

In [ ]:
s3.fillna(s3.mean())

#### Series with a date / datetime index (timeseries)

Series can have indexes representing dates / times 

In [ ]:
a

In [ ]:
a.index = pd.date_range(start='2014-1-1', periods=len(a)) # default 'period' is daily

In [ ]:
a.head()

In [ ]:
a.index

In [ ]:
### but you can convert it to an numpy array of python datetime objects if you want
py_datetimes = a.index.to_pydatetime()

And a number of useful methods for manipulation of time series is exposed

In [ ]:
### resample daily time-series to 5 minutes 'period', using forward filling method
a.resample('5min',fill_method='ffill')

In [ ]:
a

In [ ]:
### the ```shift``` method makes it easy e.g. to compare series with lead / lags 
a.shift(periods=-1)

In [ ]:
a

In [ ]:
### and the ```truncate`` method allows easy selection of time-slices
a.truncate(after='2014-1-2')

### DataFrames

**DataFrames** are IMHO one of the most powerful data structures in the Python / data analysis world. 

They can be viewed as a *collection* of named Series. They feature two **indexes**, respectively for the rows and the columns, and can contain heteregoneous data types (although it must be consistent *within* each column). 
Note that a DataFrame index, either along the rows or the columns (or both !) can contain more than one level, they are called **hierarchical indexes** and allows the representation of complex data organisation. 

If the index along the rows of a DataFrame is of **datetime** type, all the methods exposed for the Series (re-sampling, shifting, truncating, etc) are available for the DataFrame.

#### DataFrame constructions

In [ ]:
import string # part of the standard library
idx = list(string.lowercase[:10])
print(idx)

In [ ]:
df = pd.DataFrame(np.arange(100).reshape(10,10),\
                  columns=idx,index=np.arange(1,11))

In [ ]:
df

In [ ]:
### here I am creating a DataFrame from a dictionnary

df = pd.DataFrame({'A' : np.random.random(5),\
                   'B' : np.random.random(5),\
                   'C': np.random.random(5)}, index=np.arange(1,6))
print df

In [ ]:
df

#### Indexing a DataFrame

different ways, return either a Series or a DataFrame

In [ ]:
type(df['A'])

In [ ]:
df[['A']]

In [ ]:
type(df['A']) 

In [ ]:
type(df[['A']])

To access a particular *row* instead of a column, you use the *ix* method

In [ ]:
df.ix[3]

In [ ]:
df

access values per **positional indices** only

In [ ]:
df.iloc[2,0:2]

In [ ]:
type(_)

And you can combine of course row (with ix) and column indexing, using the same convention for slices as we saw for the Series 

In [ ]:
df.ix[3]['A':'B']

In [ ]:
df.ix[3][['A','C']]

you can also use the `loc` method, giving it both row **AND** columns indexes 
(the indexing is based on the *label*, **NOT** on the *position*)

In [ ]:
df

In [ ]:
df.loc[[3,5],['A','B']]

conditional indexing or subsetting of a DataFrame 

In [ ]:
df

In [ ]:
subset = df[df['A'] <= 0.7]

In [ ]:
subset

#### queries

as from Pandas version 0.14, you can build complex *database-like* queries on DataFrames

In [ ]:
df

In [ ]:
df.query('A > B')

In [ ]:
df.query('(A > B) & (B > C)')

#### Extending a DataFrame

Adding a column is easy 

In [ ]:
df

In [ ]:
df['D'] = np.random.random(5)

In [ ]:
df

The following works because Pandas understands that a single value must be repeated over the row length

In [ ]:
df['E'] = 5

In [ ]:
df

The following doesn't work because there's no way to tell **where** to insert the missing value (align to 1st or last index ?)

In [ ]:
df['F'] = np.random.random(4)

Unless we make a series out of it, with a index matching at least partly the DataFrame (row) index

In [ ]:
df['F'] = pd.Series(np.random.random(4), index=np.arange(1,5)) #

In [ ]:
df

#### Useful DataFrames methods

##### applying a numpy function

In [ ]:
df.apply(np.sqrt) # or np.sqrt(df)

##### summarizing data

In [ ]:
df.describe().T

##### replacing values

In [ ]:
df.head()

In [ ]:
df['E'].replace(3, 0.0, inplace=True) 
# you can use dictionnary to multiple replacements

In [ ]:
df

##### assign 

**Note**: `assign` is a `DataFrame` method available in version 0.16 of Pandas, to update: 
        
```
ᐅ conda update pandas 
```

In [ ]:
df.assign(F = df['A'] + df['E'])

that allows you to write very succint code

In [ ]:
df.query('A > 0.6').assign(F = df['A'] + df['E'])

#### DataFrame high-level plotting functions

In [ ]:
df.plot(ylim=[0, 6]);

In [ ]:
df.drop('E', axis=1).plot(figsize=(8,12), \
                          subplots=True, \
                          sharex=True, \
                          kind='bar', rot=0); 

In [ ]:
import pandas as pd
from sklearn.datasets import load_iris
 
iris = load_iris()
 
iris = pd.DataFrame(iris.data, columns=iris.feature_names)

In [ ]:
iris.head()

In [ ]:
iris.hist(); 

In [ ]:
from pandas.tools.plotting import scatter_matrix

In [ ]:
scatter_matrix(iris, alpha=0.4, figsize=(10, 10), diagonal='kde'); 

In [ ]:
HTML('<iframe src=http://pandas.pydata.org/pandas-docs/stable/visualization.html width=1000 height=350></iframe>')

### Input and Output in pandas

Pandas has **very powerful IO methods**, allowing to load csv, excel, tab-delimited files very easily. Pandas DataFrames can also be 
saved also in csv, excel files. 

Other supported file types are: 

* JSON (JavaScript Object Notation)
* HDF5 
* HTML (to e.g. read tables contained in HTML documents)
* SQL (Pandas can **query directly from SQL databases** thanks to [SQLAlchemy](http://www.sqlalchemy.org/))
* ...

In [ ]:
SOI = pd.read_csv('../data/NIWA_SOI.csv')

In [ ]:
SOI.head()

In [ ]:
SOI = pd.read_csv('./data/NIWA_SOI.csv', index_col=0)

In [ ]:
SOI.head()

In [ ]:
SOI.ix[1950:2000].head()

#### Stacking 

In [ ]:
SOIs = SOI.stack()

In [ ]:
SOIs.head()

In [ ]:
SOIs.index

In [ ]:
from dateutil import parser

In [ ]:
dateindex = [parser.parse("-".join(map(str, [x[0], x[1], 1]))) for x in SOIs.index]

In [ ]:
SOIs.index=dateindex

In [ ]:
SOIs.head()

In [ ]:
SOIs.plot(figsize=(12,5))

In [ ]:
pd.rolling_mean(SOIs,12).plot()

##### Saving in csv, excel etc

In [ ]:
SOIs = pd.DataFrame(SOIs)

In [ ]:
SOIs.to_excel('./data/SOI_time_series.xlsx', header=False)

In [ ]:
#!open ./data/SOI_time_series.xlsx

#### Reading Excel files

first example: an Excel file, containing one sheet, with an Excel `dates` column

In [ ]:
#!open ./data/ISO_datetime.xls

In [ ]:
data = pd.read_excel('./data/ISO_datetime.xls', sheetname='Sheet1')

In [ ]:
data.head()

In [ ]:
data.date

In [ ]:
data = pd.read_excel('../data/ISO_datetime.xls', \
                     sheetname='Sheet1', index_col=0)

In [ ]:
data.index

In [ ]:
data

#### correctly parsing date(times) when date and time information are contained in different columns

In [ ]:
!open ../data/year_month_day.xlsx

In [ ]:
ymd = pd.read_excel('../data/year_month_day.xlsx', sheetname='Sheet1')
ymd.head()

In [ ]:
ymd = pd.read_excel('../data/year_month_day.xlsx', sheetname='Sheet1', \
                    parse_dates = {'date': [0, 1, 2]}, \
                    index_col='date')
ymd.head()

In [ ]:
myd = pd.read_excel('../data/year_month_day.xlsx', sheetname='Sheet2'); myd.head()

In [ ]:
myd = pd.read_excel('../data/year_month_day.xlsx', sheetname='Sheet2',\
                    parse_dates = {'date': [1, 0, 2]}, \
                    index_col='date'); 
myd.head()

#### dealing with separators in text files

In [ ]:
HTML('<iframe src=http://www.jamstec.go.jp/frcgc/research/d1/iod/DATA/emi.weekly.txt width=900 height=200></iframe>')

You can use [regular expressions](http://en.wikipedia.org/wiki/Regular_expression) to specify what delimiters to use. 

To know more about regular expressions and their use from Python via the [re](https://docs.python.org/2/library/re.html) package, read: [http://www.ucs.cam.ac.uk/docs/course-notes/unix-courses/PythonRE/files/PythonRE.pdf](http://www.ucs.cam.ac.uk/docs/course-notes/unix-courses/PythonRE/files/PythonRE.pdf)

In [ ]:
data_weekly = pd.read_table('http://www.jamstec.go.jp/frcgc/research/d1/iod/DATA/emi.weekly.txt', \
                            header=None, sep=r'[:, \s*]', parse_dates={'date':[0,1,2]}, engine='python',\
                            index_col='date', usecols=[0,1,2,4])

In [ ]:
data_weekly.head()

In [ ]:
data_weekly.plot(legend=False);

### groupby operations in Pandas 

The **groupby** method is a very powerful method of pandas DataFrames, in a nutschell it allows you to

1. **split** your data according to unique values of a variable (or unique *combinations* of *N* variables)

2. **apply** some operation to the groups thus defined, either an *aggregation* or *transformation* method 

3. **combine** the results into a DataFrame

This process is illustrated below, where the operation is here calculating the mean of the groups's values

A very nice explanation of the **groupby** method, with examples, is available from Pandas's documentation at:  

[http://pandas.pydata.org/pandas-docs/stable/groupby.html](http://pandas.pydata.org/pandas-docs/stable/groupby.html)

and a short tutorial on Wes McKinney's blog [here](http://wesmckinney.com/blog/?p=125)

In [ ]:
Image(filename='images/split-apply-combine.png', width=800)

In [ ]:
url = "ftp://ftp.cpc.ncep.noaa.gov/wd52dg/data/indices/ersst3b.nino.mth.81-10.ascii"

In [ ]:
!wget -P ./data ftp://ftp.cpc.ncep.noaa.gov/wd52dg/data/indices/ersst3b.nino.mth.81-10.ascii

In [ ]:
data = pd.read_table('./data/ersst3b.nino.mth.81-10.ascii', sep='\s+') #the '\s+' is a regular expression meaning any number of spaces

In [ ]:
# if the network confirguration allows it, you can read directly off the URL (ftp):

#data = pd.read_table(url, sep='\s+')

In [ ]:
data.tail()

I only keep the raw - monthly - values of NINO 3.4 

In [ ]:
nino = data[['YR','MON','NINO3.4']]

In [ ]:
nino.tail()

Now I want to calculate a climatology (over the whole period available)

I first group by UNIQUE values of the variable months, I should get 12 groups

In [ ]:
groups = nino.groupby('MON')

In [ ]:
for month, group in groups:
    print month
    print group.head()

In [ ]:
climatology = groups.mean()

Same as 

    climatology = groups.aggregate(np.mean)
    
    

In [ ]:
climatology['NINO3.4'].head(12)

In [ ]:
f, ax = plt.subplots()

climatology['NINO3.4'].plot(ax=ax, kind='bar',ylim=[26,28], rot=0, width=0.9)

ax.set_xticklabels(list('JFMAMJJASOND'));

f.savefig('./climatology.pdf')

Now suppose we want to apply a function that doesnt **aggregate** the values in the groups (such as *sum*, or *mean*) but rather want to **apply a function** to those values ... 

An example would be calculating the standardized anomalies per month (to each value subtract the mean of the corresponding month, then divide by the standard-deviation)

In [ ]:
def zscore(x): 
    z = (x - x.mean()) / x.std()
    return z

In [ ]:
transformed = nino.groupby(nino.MON).apply(zscore)

In [ ]:
transformed['NINO3.4'].plot()

Now we want calculate (just) the anomalies WRT to the climatology, but with a ** *normal* established over 1980 - 2010**

In [ ]:
nino.head()

we can make use of the *query* method of Pandas DataFrames to select the climatological period

In [ ]:
nino.query('YR >= 1981 & YR <= 2000').head()

In [ ]:
def demean(x): 
    z = x - x.query('YR >= 1981 & YR <= 2000').mean()
    return z

In [ ]:
anoms = nino.groupby([nino.MON]).apply(demean)

In [ ]:
f, ax = plt.subplots(figsize=(10,6))
anoms['NINO3.4'].plot()
data['ANOM.3'].plot() # Should be the same

### Exercise

The file [Daily_clim_data_Auckland.csv](../data/Daily_clim_data_Auckland.csv) in the `data` directory contains 
daily values for rainfall, minimum and maximum temperatures for Auckland from 1972 to now.  

1. Read the data in a Pandas DataFrame, correctly setting up the index as a datetime object 
2. calculate monthly means for the temperature data, monthly totals for rainfall 
3. calculate and plot a climatology over the period 1981 - 2000 for all parameters 
4. calculate the anomalies WRT this climatology for all parameters 
5. plots the time-series of minimum and maximum temperature 
6. dig into the Scipy documentation to find how to calculate the linear trend over the period for the temperature data 
7. calculate and plot the trend 
8. Contact the [New Zealand Climate Science Coalition](http://www.nzclimatescience.org/)